# Carga del dataset Fine Persona de Hugging Face

In [1]:
!pip install huggingface_hub[hf_transfer]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 23.9 MB/s eta 0:00:00


In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [3]:
!export HF_HUB_ENABLE_HF_TRANSFER=1

In [4]:
from datasets import load_dataset

In [5]:
ds = load_dataset("argilla/FinePersonas-v0.1-clustering-100k", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/37.1k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/350M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/350M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [6]:
ds

Dataset({
    features: ['id', 'persona', 'model_name_embeddings', 'embedding', 'nn_indices', 'nn_scores', 'projection', 'cluster_label', 'summary_label'],
    num_rows: 100000
})

Convierte ds a dataframe de pandas con la columna id, persona, cluster_label y summary_label

In [7]:
import pandas as pd

df = pd.DataFrame({
    'id': ds['id'],
    'persona': ds['persona'],
    'cluster_label': ds['cluster_label'],
    'summary_label': ds['summary_label']
})

In [8]:
df.head()

,id,persona,cluster_label,summary_label
0,<urn:uuid:4cda0919-3bce-43ce-aa46-6c205b242490>,A geography teacher or high school educator fo...,-1,"""None"""
1,<urn:uuid:2bdad496-891e-44f0-8041-49f9a381a88a>,A meteorologist or a radar technician dealing ...,0,"[""Environmental"", ""Scientific"", ""Professional""]"
2,<urn:uuid:9e0159f8-2478-4fc2-818a-e325f1b1e9ac>,A public health specialist focused on childhoo...,0,"[""Environmental"", ""Scientific"", ""Professional""]"
3,<urn:uuid:b30c1d38-869e-495d-a574-8b702f29522c>,An English as a second language (ESL) teacher ...,1,"[""Education"", ""Academia"", ""Specialized Experti..."
4,<urn:uuid:fdddfb80-360e-4cd9-8cbd-583976fe9520>,An academic or educator in the field of econom...,2,"[""Academic"", ""Economics"", ""Finance""]"


Se obtiene una muestra para ejemplificar los casos de uso

In [27]:
sample = df['persona'].sample(15)

In [28]:
sample = pd.DataFrame(sample)

In [10]:
sample.iloc[0]

'An instructional designer or educator focused on mathematics, problem-solving, and critical thinking, likely someone who teaches or develops educational resources for students or professionals on effective problem-solving strategies and techniques.'

# Conexión API OpenAI

In [11]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


In [12]:
import openai

client = openai.OpenAI(api_key='Tu_Api_key')

In [35]:
def obtener_respuesta(prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # Ajusta el modelo según lo que necesites
        messages=[
            {"role": "system", "content": """
            You're a helpful assistant that responds in spanish in 100 words, straight to the point
            """},
            {"role": "user", "content": prompt}
        ]
    )
    output = response.choices[0].message.content
    return output

In [14]:
PROMPT_TEMPLATE_1 = """
Based on the following persona, generate a list of content recommendations for a streaming platform,
focusing on documentaries, shows or movies that may interest them:
"""

### Investigación de Mercado e Insights del Consumidor

Para las empresas, comprender las preferencias del cliente antes de lanzar un producto o campaña de marketing es invaluable. FinePersonas puede simular diversos segmentos de consumidores, ofreciendo información sobre cómo podrían comportarse y qué productos prefieren. Este enfoque puede refinar significativamente los diseños de productos, estrategias publicitarias e incluso el posicionamiento de marcas.

**Prompt en español:** Simula cómo el siguiente segmento de consumidores reaccionaría ante el lanzamiento de un nuevo producto de tecnología wearable (por ejemplo, un reloj inteligente) que monitorea la salud, enfocado en la prevención de enfermedades y bienestar:

In [15]:
PROMPT_TEMPLATE_2 = """
Simulate how the following consumer segment would react to the launch of a new wearable technology product
(e.g., a smartwatch) that monitors health, focusing on disease prevention and wellness:
"""

In [16]:
import pandas as pd
from tqdm import tqdm

# Asumiendo que ya tienes definida la función obtener_respuesta y el cliente

def procesar_descripciones(df, prompt_template):
    resultados = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Procesando personas"):
        persona = row['persona']
        prompt = prompt_template.format(persona=persona)

        try:
            output = obtener_respuesta(prompt)
            resultados.append({'persona': persona, 'output': output})
        except Exception as e:
            print(f"Error procesando persona: {persona}. Error: {str(e)}")
            resultados.append({'persona': persona, 'output': 'Error en el procesamiento'})

    return pd.DataFrame(resultados)

In [36]:
# Uso de la función
prompt_template = """
Simulate how the following consumer segment would react to the launch of a new wearable technology product
(e.g., a smartwatch) that monitors health, focusing on disease prevention and wellness:
{persona}
"""

# Asumiendo que ya tienes tu DataFrame 'df' con la columna 'persona'
df_resultados = procesar_descripciones(sample, prompt_template)


Procesando personas: 100%|██████████| 15/15 [00:25<00:00,  1.69s/it]


In [44]:
df_resultados.head()

,persona,output
0,A science writer or astronomy journalist focus...,"Un periodista científico, especializado en ast..."
1,A botanist or plant biologist with a focus on ...,Un botánico o biólogo vegetal podría reacciona...
2,A pediatrician or child development specialist...,Un pediatra o especialista en desarrollo infan...
3,A space historian or a science educator who is...,Un historiador del espacio o un educador cient...
4,A computer architecture student or an embedded...,Un estudiante de arquitectura de computadoras ...


In [38]:

# Exportar el DataFrame a un archivo Excel
df_resultados.to_excel('df_resultados.xlsx', index=False)
